# T1574.009 - Path Interception by Unquoted Path
Adversaries may execute their own malicious payloads by hijacking vulnerable file path references. Adversaries can take advantage of paths that lack surrounding quotations by placing an executable in a higher level directory within the path, so that Windows will choose the adversary's executable to launch.

Service paths (Citation: Microsoft CurrentControlSet Services) and shortcut paths may also be vulnerable to path interception if the path has one or more spaces and is not surrounded by quotation marks (e.g., <code>C:\unsafe path with space\program.exe</code> vs. <code>"C:\safe path with space\program.exe"</code>). (Citation: Help eliminate unquoted path) (stored in Windows Registry keys) An adversary can place an executable in a higher level directory of the path, and Windows will resolve that executable instead of the intended executable. For example, if the path in a shortcut is <code>C:\program files\myapp.exe</code>, an adversary may create a program at <code>C:\program.exe</code> that will be run instead of the intended program. (Citation: Windows Unquoted Services) (Citation: Windows Privilege Escalation Guide)

This technique can be used for persistence if executables are called on a regular basis, as well as privilege escalation if intercepted executables are started by a higher privileged process.

## Atomic Tests

### Atomic Test #1 - Execution of program.exe as service with unquoted service pathWhen a service is created whose executable path contains spaces and isn’t enclosed within quotes, leads to a vulnerability
known as Unquoted Service Path which allows a user to gain SYSTEM privileges.
In this case, if an executable program.exe in C:\ exists, C:\program.exe will be executed instead of test.exe in C:\Program Files\subfolder\test.exe.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
copy PathToAtomicsFolder\T1574.009\bin\WindowsServiceExample.exe "C:\Program Files\windows_service.exe"
copy PathToAtomicsFolder\T1574.009\bin\WindowsServiceExample.exe "C:\program.exe"
sc create "Example Service" binpath= "C:\Program Files\windows_service.exe" Displayname= "Example Service" start= auto
sc start "Example Service"
```

In [ ]:
Invoke-AtomicTest T1574.009 -TestNumbers 1

#### Cleanup: 
```cmd
sc stop "Example Service" >nul 2>&1
sc delete "Example Service" >nul 2>&1
del "C:\Program Files\windows_service.exe" >nul 2>&1
del "C:\program.exe" >nul 2>&1
del "C:\Time.log" >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1574.009 -TestNumbers 1 -Cleanup

## Detection
Monitor file creation for files named after partial directories and in locations that may be searched for common processes through the environment variable, or otherwise should not be user writable. Monitor the executing process for process executable paths that are named for partial directories. Monitor file creation for programs that are named after Windows system programs or programs commonly executed without a path (such as "findstr," "net," and "python"). If this activity occurs outside of known administration activity, upgrades, installations, or patches, then it may be suspicious.

Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as network connections made for Command and Control, learning details about the environment through Discovery, and Lateral Movement.